In [6]:
#라이브러리 임포트

import numpy as np

import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [7]:


env = GridworldEnv()

In [8]:
#Policy evaluation을 먼저 만들어준다. 

def policy_eval(policy,env,discount_factor=1.0, theta=0.00001):
    
    V=np.zeros(env.nS)
    
    while True:
        delta=0
        for s in range(env.nS):
            v=0
            for a, action_prob in enumerate(policy[s]):#인덱스와 내용 출력
                
                for prob,next_state,reward,done in env.P[s][a]:
                    v+= action_prob*prob*(reward+discount_factor*V[next_state])
            delta=max(delta, np.abs(v-V[s]))
            V[s]=v
            
        if delta<theta:
            break
    return np.array(V)
        
            


In [15]:
#Policy 개선하기


def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
     Policy Improvement 알고리즘. 
     최적 폴리시를 찾을때까지 폴리시를 관찰하고 개선한다. 
    
    Args:
        env: The OpenAI 환경
        policy_eval_fn: 폴리시 이벨류에이션 함수는 3개의 아규먼트를 받는다. 
        1) 폴리시 2) 환경 3)디스카운트펙터
        discount_factor: Lambda discount factor.
        
    Returns:1
        폴리시와 벨류를 포함한 터플을 리턴
        폴리시는 최적화된 폴리시이다. 각각 스테이트의 [스테이트, 엑션]의 모양이다. 
        엑션의 확률분포가 포함되어있다.
        V 는 최적 폴리시를 위한 벨류함수이다.
        
    """
    # 랜덤 폴리시로 시작
    policy = np.ones([env.nS,env.nA])/env.nA #각 엑션을 0.25로 
    while True: 
        # 현재 policy evaluate 
        
        V= policy_eval_fn(policy,env,discount_factor)
        
        
        #폴리시에 변화를 주면 false로 설정된다. 
        policy_stable = True

        
        
        #각 스테이트마다.. 
        for s in range(env.nS):
            
            #폴리시를 가장 크게 하는 값을 찾는다
            chosen_a = np.argmax(policy[s])
            
            # Find the best action by one-step lookahead
            #한 스텝 다음에서의 베스트엑션을 찾는다. 
        
                
            action_values = np.zeros(env.nA) #엑션만큼의 값이 0인 어레이를 생성함
            
            #각 스테이트에서 할수 있는 엑션에서 
            for a in range(env.nA):
                #엑션의 확률과, 엑션을 했을때 갈수 있는 다음스테이트, 리워드, 그리고 끝 ( env.P[s][a] 안에서 ) 
                for prob, next_state, reward, done in env.P[s][a]:
                    
                    #엑션의 가치를 구한다. 
                    action_values[a] += prob * (reward + discount_factor * V[next_state])
                    
                    #np.argmax를 사용하여 action_values
            best_a = np.argmax(action_values)
            
            
            
            # Greedily update the policy
            if chosen_a != best_a:
                policy_stable = False
                
            # 2-D array with ones on the diagonal and zeros 4*4인  ones on the diagonal and zeros elsewher를 만들어주고,
            
            #해주고 베스트엑션을 리턴한다. 
            
            policy[s] = np.eye(env.nA)[best_a]
        
        # If the policy is stable we've found an optimal policy. Return it
        
        if policy_stable:
            return policy, V

In [17]:
%%timeit -n 1000
policy, v = policy_improvement(env)

"""
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")
"""




11.2 ms ± 852 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
%%timeit

v

15.9 ns ± 0.567 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)
